# Limpieza de datos

In [65]:
sudo apt install tensorflow

SyntaxError: invalid syntax (2612498661.py, line 1)

In [63]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

Carga de Datos

In [38]:
df = pd.read_csv('../data/training.csv')
print(df.shape)
df.head()

(11696, 15)


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-11-19,S005,P0020,Furniture,South,452,432,176,442.08,84.09,0,Snowy,1,85.46,Summer
1,2023-06-29,S005,P0006,Clothing,South,377,212,31,221.02,50.49,15,Sunny,1,46.74,Autumn
2,2022-03-10,S005,P0010,Furniture,South,420,38,185,46.02,27.34,5,Sunny,1,22.62,Spring
3,2022-02-07,S005,P0001,Electronics,South,92,9,32,14.67,88.84,10,Snowy,1,85.76,Summer
4,2022-06-23,S005,P0015,Furniture,West,272,250,178,268.00,98.73,20,Sunny,1,96.54,Summer


In [39]:
# Revisar serie temporal valida

In [40]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df = df.sort_index()


In [41]:
df.head()

,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
Date,,,,,,,,,,,,,,
2022-01-01,S005,P0010,Clothing,East,87,8,141,14.38,37.04,10,Rainy,1,36.87,Summer
2022-01-01,S005,P0018,Furniture,West,161,107,66,103.97,48.96,10,Snowy,1,50.03,Spring
2022-01-01,S005,P0011,Groceries,West,427,104,128,94.65,54.12,10,Rainy,1,53.03,Summer
2022-01-01,S005,P0002,Clothing,West,415,298,107,292.96,57.15,5,Snowy,1,60.71,Autumn
2022-01-01,S005,P0005,Clothing,West,180,21,47,38.63,92.86,15,Rainy,1,96.30,Winter


# Proceso de limpieza

In [46]:
df.columns

Index(['Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level',
       'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount',
       'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing',
       'Seasonality'],
      dtype='object')

In [58]:
def preprocesar_limpieza_lstm(df):
    df = df.copy()

    # Asegurar que Date exista y sea datetime
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
    else:lqr(df, columns):
        for col in columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            df = df[(df[col] >= lower) & (df[col] <= upper)]
        return df

    numeric_cols = ['Units Sold', 'Demand Forecast', 'Price', 'Competitor Pricing']
    df = remove_outliers_iqr(df, numeric_cols)

    # Normalizar columnas numéricas
    scaler = MinMaxScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Convertir columna booleana
    if 'Holiday/Promotion' in df.columns:
        df['Holiday/Promotion'] = df['Holiday/Promotion'].astype(bool)

    # Extraer componentes de fecha
    if pd.api.types.is_datetime64_any_dtype(df['Date']):
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek

    return df


In [59]:
df

,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
Date,,,,,,,,,,,,,,
2022-01-01,S005,P0010,Clothing,East,87,0.018433,141,0.054504,0.300367,10,Rainy,True,0.315879,Summer
2022-01-01,S005,P0018,Furniture,West,161,0.246544,66,0.257541,0.432826,10,Snowy,True,0.448141,Spring
2022-01-01,S005,P0011,Groceries,West,427,0.239631,128,0.236419,0.490166,10,Rainy,True,0.478291,Summer
2022-01-01,S005,P0002,Clothing,West,415,0.686636,107,0.685847,0.523836,5,Snowy,True,0.555477,Autumn
2022-01-01,S005,P0005,Clothing,West,180,0.048387,47,0.109462,0.920658,15,Rainy,True,0.913166,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01,S005,P0013,Electronics,West,389,0.635945,99,0.650289,0.244916,0,Snowy,False,0.306131,Summer
2024-01-01,S005,P0010,Toys,North,308,0.423963,94,0.444578,0.102122,0,Cloudy,False,0.175377,Winter
2024-01-01,S005,P0003,Furniture,South,98,0.211982,118,0.261258,0.157351,5,Cloudy,False,0.177186,Winter


In [62]:
# LSTM multivariado, realizando una prediccion de 30 dias